In [0]:
import nltk

In [3]:
nltk.chat.chatbots()

Which chatbot would you like to talk to?
  1: Eliza (psycho-babble)
  2: Iesha (teen anime junky)
  3: Rude (abusive bot)
  4: Suntsu (Chinese sayings)
  5: Zen (gems of wisdom)

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatbot number

Enter a number in the range 1-5:     Error: bad chatb

KeyboardInterrupt: ignored